In [ ]:
# 推箱子的过程中，人能够

In [340]:
from collections import deque
import copy

class Solution:
    def minPushBox(self, grid):
        c_grid = copy.deepcopy(grid)
        def get_man_pos(grid):
            # 1、man_arrived：查看人可以走到哪些位置
            can_reach = set()
            for r in range(rows):
                for c in range(cols):
                    if grid[r][c] == 'S':
                        man_pos = (r, c)
                        stack = [(r, c)] # dfs
                        while stack:
                            x, y = stack.pop()
                            can_reach.add((x, y))
                            for nr, nc in (x+1, y), (x-1, y), (x, y+1), (x, y-1):
                                if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.' and (nr, nc) not in can_reach:
                                    can_reach.add((nr, nc))
                                    stack.append((nr, nc))
            return can_reach, man_pos
        
        rows, cols = len(grid), len(grid[0])
        can_reach, man_pos = get_man_pos(grid)
        box, man_box = [], set() # 箱子坐标，箱子附近人的坐标
        tar, man_tar = [], set() # 目标点坐标、箱子坐标、目标点附近人所在的位置
        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == 'B':
                    box = [r, c] # 箱子位置, 箱子的四个方向一定要有 '.', 得让人过去
                    for nr, nc in (r+1, c), (r-1, c), (r, c+1), (r, c-1):
                        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.':
                            man_box.add((nr, nc))
                elif grid[r][c] == 'T': # 目标点所在位置
                    tar = [r, c] # 临近目标点的周围一定要有 空格，放箱子，在箱子的后面才是人站的地方
                    for dr, dc in (1, 0), (0, 1), (-1, 0), (0, -1):
                        box_pos = []
                        nr, nc = r + dr, c + dc
                        # 临近目标的一圈肯定要是有空格的，放箱子
                        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.':
                            box_pos.append((nr, nc))
                        # 箱子的后面一定还可以放人
                        if box_pos:
                            if (dr, dc) == (1, 0) and 0 <= nr+1 < rows and grid[nr+1][nc] == '.': # 箱子在下
                                if (nr+1, nc) in can_reach:
                                    man_tar.add((nr, nc, nr+1, nc))
                            elif (dr, dc) == (0, 1) and 0 <= nc+1 < cols and grid[nr][nc+1] == '.': # 右
                                if (nr, nc+1) in can_reach:
                                    man_tar.add((nr, nc, nr, nc+1))
                            elif (dr, dc) == (-1, 0) and 0 <= nr-1 < rows and grid[nr-1][nc] == '.': # 上
                                if (nr-1, nc) in can_reach:
                                    man_tar.add((nr, nc, nr-1, nc))
                            elif (dr, dc) == (0, -1) and 0 <= nc-1 < cols and grid[nr][nc-1] == '.': # 左
                                if (nr, nc-1) in can_reach:
                                    man_tar.add((nr, nc, nr, nc-1))
                if box and tar: #　初始化完成，退出
                    break
        if not man_box or not man_tar: # 人无法站在箱子周围或者目标点周围，返回-1
            return -1
   
        # 根据之前获得的条件进行初始化
        dq = deque() # 箱子和人的起始位置
        best_state = {}
        for x, y in man_box:
            dq.append(box + [x, y, 0])
            best_state[(box[0], box[1], x, y)] = 0 # 到达每个位置的最少移动步数
        
        space = ('T', '.') # 箱子可以推到的地方
        # can_reach = set() # can_reach的值是随着箱子和人的位置是会变换的
        # 开始推箱子
        while dq:
            for _ in range(len(dq)):
                br, bc, mr, mc, cnt = dq.popleft() # 箱子、人在grid中的位置
                if [br, bc] == tar:
                    return cnt
                # 从人和箱子的位置，判断箱子是往哪里推的
                if br == mr:
                    direc = 'R' if bc > mc else 'L'
                else:
                    direc = 'D' if br > mr else 'U'
                
                grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = '.', '.'
                grid[br][bc], grid[mr][mc] = 'B', 'S'
                # 不同的方向推动箱子, 
                if direc == 'L':
                    if 0 <= bc-1 < cols and grid[br][bc-1] in space:
                        state = (br, bc-1, br, bc)
                        if state not in best_state or cnt+1 < best_state[state]:
                            best_state[state] = cnt + 1
                            dq.append((br, bc-1, br, bc, cnt+1))
                        
                            grid[br][bc], grid[mr][mc] = '.', '.'
                            # 达到下一个位置之后，就需要探索，当前箱子位置的四周其他位置上能不能站人
                            grid[br][bc], grid[br][bc-1] = 'S', 'B'
                            can_reach, _ = get_man_pos(grid)

                            for dr, dc in (1, 0), (0, 1), (-1, 0), (0, -1):
                                nr, nc = br+dr, bc-1+dc # 处于新位置的箱子周围人的位置
                                if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.':
                                    n_state = (br, bc-1, nr, nc) # 下一个位置上人站的位置
                                    if (nr, nc) in can_reach and (n_state not in best_state or cnt + 1 < best_state[n_state]):
                                        best_state[n_state] = cnt + 1
                                        dq.append((br, bc-1, nr, nc, cnt+1))
                            grid[br][bc], grid[br][bc-1] = '.', '.'
                            grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = 'S', 'B'
                elif direc == 'R':
                    if 0 <= bc+1 < cols and grid[br][bc+1] in space:
                        state = (br, bc+1, br, bc)
                        if state not in best_state or cnt+1 < best_state[state]:
                            best_state[state] = cnt + 1
                            dq.append((br, bc+1, br, bc, cnt+1))
                            
                        grid[br][bc], grid[mr][mc] = '.', '.'
                        # 达到下一个位置之后，就需要探索，当前箱子位置的四周其他位置上能不能站人
                        grid[br][bc], grid[br][bc+1] = 'S', 'B'
                        can_reach, _ = get_man_pos(grid)
                        
                        for dr, dc in (1, 0), (0, 1), (-1, 0), (0, -1):
                            nr, nc = br+dr, bc+1+dc
                            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.' :
                                n_state = (br, bc+1, nr, nc) # 下一个位置上人站的位置
                                if (nr, nc) in can_reach and (n_state not in best_state or cnt + 1 < best_state[n_state]):
                                    best_state[n_state] = cnt + 1
                                    dq.append((br, bc+1, nr, nc, cnt+1))
                        grid[br][bc], grid[br][bc+1] = '.', '.'
                        grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = 'S', 'B'
                elif direc == 'D':
                    if 0 <= br+1 < rows and grid[br+1][bc] in space:
                        state = (br+1, bc, br, bc)
                        if state not in best_state or cnt+1 < best_state[state]:
                            best_state[state] = cnt + 1
                            dq.append((br+1, bc, br, bc, cnt+1))

                            grid[br][bc], grid[mr][mc] = '.', '.'
                            # 达到下一个位置之后，就需要探索，当前箱子位置的四周其他位置上能不能站人
                            grid[br][bc], grid[br+1][bc] = 'S', 'B'
                            can_reach, _ = get_man_pos(grid)

                            for dr, dc in (1, 0), (0, 1), (-1, 0), (0, -1):
                                nr, nc = br+1+dr, bc+dc
                                if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.':
                                    n_state = (br+1, bc, nr, nc) # 下一个位置上人站的位置
                                    if (nr, nc) in can_reach and (n_state not in best_state or cnt+1 < best_state[n_state]):
                                        best_state[n_state] = cnt + 1
                                        dq.append((br+1, bc, nr, nc, cnt+1))
                            grid[br][bc], grid[br+1][bc] = '.', '.'
                            grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = 'S', 'B'
                elif direc == 'U':
                    if 0 <= br-1 < cols and grid[br-1][bc] in space:
                        state = (br-1, bc, br, bc)
                        if state not in best_state or cnt+1 < best_state[state]:
                            best_state[state] = cnt + 1
                            dq.append((br-1, bc, br, bc, cnt+1))
                            
                            grid[br][bc], grid[mr][mc] = '.', '.'
                            # 达到下一个位置之后，就需要探索，当前箱子位置的四周其他位置上能不能站人
                            grid[br][bc], grid[br-1][bc] = 'S', 'B'
                            can_reach, _ = get_man_pos(grid)

                            for dr, dc in (1, 0), (0, 1), (-1, 0), (0, -1):
                                nr, nc = br-1+dr, bc+dc
                                if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == '.':
                                    n_state = (br-1, bc, nr, nc) # 下一个位置上人站的位置
                                    if (nr, nc) in can_reach and (n_state not in best_state or cnt+1 < best_state[n_state]):
                                        best_state[n_state] = cnt + 1
                                        dq.append((br-1, bc, nr, nc, cnt+1))
                            grid[br][bc], grid[br-1][bc] = '.', '.'
                            grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = 'S', 'B'
                grid[br][bc], grid[mr][mc] = '.', '.'
                grid[man_pos[0]][man_pos[1]], grid[box[0]][box[1]] = 'S', 'B'
        return -1

In [341]:
solution = Solution()
solution.minPushBox([[".","#",".",".",".",".","#",".",".","#",".","."],
                     [".",".",".","#",".","#",".",".","#","#",".","."],
                     [".",".",".",".",".",".",".",".",".",".",".","."],
                     [".",".",".",".",".",".",".",".",".",".",".","."],
                     [".","B",".",".",".","#","#",".",".","#",".","."],
                     [".","#","#","#",".",".",".",".",".",".",".","#"],
                     [".",".",".",".",".",".",".",".",".",".","#","#"],
                     [".",".",".","#","#",".","#",".",".",".",".","#"],
                     ["#",".","#",".",".","T",".",".",".",".",".","."],
                     [".","#",".",".",".",".","#",".",".",".",".","."],
                     [".",".","S",".",".",".",".",".",".","#",".","."],
                     ["#",".",".",".",".","#",".",".",".",".","#","."]])

8

In [ ]:
[['.', '#', '.', '.', '.', '.', '#', '.', '.', '#', '.', '.'], 
 ['.', '.', '.', '#', '.', '#', '.', '.', '#', '#', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '#', '#', '.', '.', '#', '.', '.'], 
 ['.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '.', '#'], 
 ['.', '.', '.', 'S', 'B', '.', '.', '.', '.', '.', '#', '#'], 
 ['.', '.', '.', '#', '#', '.', '#', '.', '.', '.', '.', '#'], 
 ['#', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.'], 
 ['#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '#', '.']]

In [ ]:
[['.', '#', '.', '.', '.', '.', '#', '.', '.', '#', '.', '.'], 
 ['.', '.', '.', '#', '.', '#', '.', '.', '#', '#', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '#', '#', '.', '.', '#', '.', '.'], 
 ['.', '#', '#', '#', '.', '.', '.', '.', '.', '.', '.', '#'], 
 ['.', '.', '.', 'S', 'B', '.', '.', '.', '.', '.', '#', '#'], 
 ['.', '.', '.', '#', '#', '.', '#', '.', '.', '.', '.', '#'], 
 ['#', '.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 
 ['.', '#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'], 
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.', '.'], 
 ['#', '.', '.', '.', '.', '#', '.', '.', '.', '.', '#', '.']]

In [294]:
[(3, 5, 2, 5, 1), (3, 5, 4, 5, 1), (3, 5, 3, 6, 1), (3, 5, 3, 4, 1), (1, 5, 2, 5, 1)]
[(4, 5, 3, 5, 2), (4, 5, 3, 5, 2), (3, 4, 3, 5, 2), (3, 4, 3, 3, 2), (3, 6, 3, 5, 2), (3, 6, 2, 6, 2), (3, 6, 3, 5, 2)]
[(3, 3, 3, 4, 3), (3, 3, 4, 3, 3), (3, 3, 2, 3, 3), (3, 3, 3, 2, 3)]
[(3, 2, 3, 3, 4), (3, 2, 4, 2, 4), (3, 2, 2, 2, 4), (3, 2, 3, 1, 4), (2, 3, 3, 3, 4), (4, 3, 3, 3, 4), (4, 3, 3, 3, 4), (4, 3, 4, 2, 4), (3, 4, 3, 3, 4)]
[(3, 1, 3, 2, 5), (3, 1, 4, 1, 5), (3, 1, 2, 1, 5), (2, 2, 3, 2, 5), (2, 2, 2, 1, 5), (4, 2, 3, 2, 5), (4, 2, 5, 2, 5), (4, 2, 4, 3, 5), (4, 2, 3, 2, 5), (4, 2, 4, 1, 5), (3, 3, 3, 2, 5), (1, 3, 2, 3, 5)]

[(3, 1, 3, 2, 5),
 (3, 1, 4, 1, 5),
 (3, 1, 2, 1, 5),
 (2, 2, 3, 2, 5),
 (2, 2, 2, 1, 5),
 (4, 2, 3, 2, 5),
 (4, 2, 5, 2, 5),
 (4, 2, 4, 3, 5),
 (4, 2, 3, 2, 5),
 (4, 2, 4, 1, 5),
 (3, 3, 3, 2, 5),
 (1, 3, 2, 3, 5)]

In [ ]:
[['#', '.', '.', '#', '#', '#', '#', '#'], 
 ['#', '.', 'B', 'S', '#', '.', '.', '#'], 
 ['#', '.', '.', '.', '#', '.', '.', '#'], 
 ['#', '.', '.', '.', '.', '.', '.', '#'], 
 ['#', '.', '.', '.', '#', '.', '.', '#'], 
 ['#', '.', '.', '#', '#', '#', '#', '#']] {(1, 3), (2, 1), (1, 6), (5, 1), (2, 5), (3, 3), (1, 5), (3, 6), (2, 2), (3, 4), (4, 1), (1, 1), (3, 2), (2, 6), (4, 5), (2, 3), (4, 2), (3, 5), (0, 1), (4, 6), (3, 1), (4, 3), (5, 2), (0, 2)}
